# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 20 2022 9:36AM,249063,21,631154,"NBTY Global, Inc.",Released
1,Oct 20 2022 9:36AM,249062,21,631153,"NBTY Global, Inc.",Released
2,Oct 20 2022 9:32AM,249059,10,0086088919,ISDIN Corporation,Released
3,Oct 20 2022 9:32AM,249059,10,0086088916,ISDIN Corporation,Released
4,Oct 20 2022 9:32AM,249059,10,0086088917,ISDIN Corporation,Released
5,Oct 20 2022 9:32AM,249059,10,0086088918,ISDIN Corporation,Released
6,Oct 20 2022 9:32AM,249059,10,0086088933,ISDIN Corporation,Released
7,Oct 20 2022 9:32AM,249059,10,0086088935,ISDIN Corporation,Released
8,Oct 20 2022 9:32AM,249059,10,0086088947,ISDIN Corporation,Released
9,Oct 20 2022 9:32AM,249059,10,0086088946,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,249059,Released,60
27,249060,Released,16
28,249061,Released,4
29,249062,Released,1
30,249063,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249059,NaN,60.0
249060,NaN,16.0
249061,NaN,4.0
249062,NaN,1.0
249063,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248896,0.0,1.0
248961,0.0,1.0
248979,0.0,2.0
248983,0.0,1.0
248984,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248896,0,1
248961,0,1
248979,0,2
248983,0,1
248984,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248896,0,1
1,248961,0,1
2,248979,0,2
3,248983,0,1
4,248984,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248896,,1
1,248961,,1
2,248979,,2
3,248983,,1
4,248984,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc."
1,Oct 20 2022 9:36AM,249062,21,"NBTY Global, Inc."
2,Oct 20 2022 9:32AM,249059,10,ISDIN Corporation
62,Oct 20 2022 9:31AM,249060,10,ISDIN Corporation
78,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation
82,Oct 20 2022 8:58AM,249056,10,Methapharm-G
83,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation
103,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd"
154,Oct 20 2022 8:40AM,249051,10,Eywa Pharma Inc.
155,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",,1
1,Oct 20 2022 9:36AM,249062,21,"NBTY Global, Inc.",,1
2,Oct 20 2022 9:32AM,249059,10,ISDIN Corporation,,60
3,Oct 20 2022 9:31AM,249060,10,ISDIN Corporation,,16
4,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,,4
5,Oct 20 2022 8:58AM,249056,10,Methapharm-G,1,
6,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation,1,19
7,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd",20,31
8,Oct 20 2022 8:40AM,249051,10,Eywa Pharma Inc.,,1
9,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,,35


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1,
1,Oct 20 2022 9:36AM,249062,21,"NBTY Global, Inc.",1,
2,Oct 20 2022 9:32AM,249059,10,ISDIN Corporation,60,
3,Oct 20 2022 9:31AM,249060,10,ISDIN Corporation,16,
4,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,4,
5,Oct 20 2022 8:58AM,249056,10,Methapharm-G,,1
6,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation,19,1
7,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd",31,20
8,Oct 20 2022 8:40AM,249051,10,Eywa Pharma Inc.,1,
9,Oct 20 2022 8:30AM,249008,10,ISDIN Corporation,35,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1,
1,Oct 20 2022 9:36AM,249062,21,"NBTY Global, Inc.",1,
2,Oct 20 2022 9:32AM,249059,10,ISDIN Corporation,60,
3,Oct 20 2022 9:31AM,249060,10,ISDIN Corporation,16,
4,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1.0,NaN
1,Oct 20 2022 9:36AM,249062,21,"NBTY Global, Inc.",1.0,NaN
2,Oct 20 2022 9:32AM,249059,10,ISDIN Corporation,60.0,NaN
3,Oct 20 2022 9:31AM,249060,10,ISDIN Corporation,16.0,NaN
4,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1.0,0.0
1,Oct 20 2022 9:36AM,249062,21,"NBTY Global, Inc.",1.0,0.0
2,Oct 20 2022 9:32AM,249059,10,ISDIN Corporation,60.0,0.0
3,Oct 20 2022 9:31AM,249060,10,ISDIN Corporation,16.0,0.0
4,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3735360,161.0,21.0
16,498092,1.0,4.0
19,996074,43.0,48.0
20,248961,1.0,0.0
21,747021,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3735360,161.0,21.0
1,16,498092,1.0,4.0
2,19,996074,43.0,48.0
3,20,248961,1.0,0.0
4,21,747021,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,161.0,21.0
1,16,1.0,4.0
2,19,43.0,48.0
3,20,1.0,0.0
4,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,161.0
1,16,Released,1.0
2,19,Released,43.0
3,20,Released,1.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Executing,21.0,4.0,48.0,0.0,0.0
Released,161.0,1.0,43.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Executing,21.0,4.0,48.0,0.0,0.0
1,Released,161.0,1.0,43.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Executing,21.0,4.0,48.0,0.0,0.0
1,Released,161.0,1.0,43.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()